In [67]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder


import sys
sys.path.append("..")
from utils.preprocess import *

import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut

In [68]:
exported_path = '../exported_models/'

In [69]:
input_directory = "../assets/Respeck"

data_list = []
label_list = []
total_rows = 0

all_occu = {}

# for folder_name in os.listdir(input_directory):
#     folder_path = os.path.join(input_directory, folder_name)
#     for file in os.listdir(folder_path):
#         if file.endswith(".csv") and "unprocessed" not in file:
            
#             filename = os.path.join(folder_path, file)
#             label, _, _ = extract_activity_and_status(filename)
#             if label not in all_occu:
#                 all_occu["label"] = 1
#                 all_occu["directory"] = [filename]
#             else:
#                 all_occu["label"] += 1
#                 all_occu["directory"].append(filename)
#             df = pd.read_csv(filename, usecols=[2,3,4])

#             # Determine the number of rows to take from this file
#             rows_to_take = min(700 - total_rows, len(df))
            
#             # Update the df to only contain the necessary rows and update our counter
#             df = df.head(rows_to_take)
#             label_list.append(label)
#             data_list.append(df)

# Define window size and overlap
window_size = 50
overlap_size = 25  # This is often set to 50% of the window size

all_occu = {}
data_windows = []

scaler = StandardScaler()

for folder_name in os.listdir(input_directory):
    folder_path = os.path.join(input_directory, folder_name)
    if folder_name == ".DS_Store":
        continue
    for file in os.listdir(folder_path):
        if file.endswith(".csv") and "unprocessed" not in file:
            
            filename = os.path.join(folder_path, file)
            label, _, _ = extract_activity_and_status(filename)
            if label not in all_occu:
                all_occu[label] = {'count': 1, 'directories': [filename]}
            else:
                all_occu[label]['count'] += 1
                all_occu[label]['directories'].append(filename)
            
            df = pd.read_csv(filename, usecols=[2, 3, 4])

            # Determine the number of rows to take from this file
            rows_to_take = min(700 - total_rows, len(df))
            
            # Update the df to only contain the necessary rows and update our counter
            df = df.head(rows_to_take)

            # Apply sliding window technique
            for start in range(0, len(df), window_size - overlap_size):
                end = start + window_size
                window = df.iloc[start:end].values  # Convert the window to a NumPy array
                if len(window) < 50:
                    continue

                # Normalize the window data
                # if not data_windows:  # If this is the first window, fit the scaler
                #     scaler.fit(window)
                # window_normalized = scaler.transform(window)  # Transform the data
                
                # # Add the normalized window and label to the lists
                data_windows.append(window)
                label_list.append(label)
# print(data_windows)

In [89]:
data_windows

[array([[-0.79125977,  0.09881592, -0.5942993 ],
        [-0.79052734,  0.09881592, -0.5935669 ],
        [-0.7976074 ,  0.09295654, -0.5984497 ],
        [-0.7734375 ,  0.09979248, -0.5852661 ],
        [-0.80029297,  0.09637451, -0.5940552 ],
        [-0.78344727,  0.10247803, -0.59454346],
        [-0.7939453 ,  0.09002686, -0.59381104],
        [-0.7907715 ,  0.09564209, -0.5869751 ],
        [-0.77978516,  0.08514404, -0.63360596],
        [-0.79125977,  0.09564209, -0.5667114 ],
        [-0.7702637 ,  0.10125732, -0.60821533],
        [-0.7858887 ,  0.10247803, -0.5886841 ],
        [-0.7910156 ,  0.1071167 , -0.59820557],
        [-0.78637695,  0.11126709, -0.58184814],
        [-0.79956055,  0.11224365, -0.5774536 ],
        [-0.78881836,  0.1005249 , -0.58306885],
        [-0.79248047,  0.11273193, -0.5967407 ],
        [-0.79711914,  0.09637451, -0.57354736],
        [-0.7890625 ,  0.11517334, -0.5835571 ],
        [-0.8029785 ,  0.11077881, -0.5757446 ],
        [-0.7883301 

In [71]:
print(len(data_windows[0]))
print(len(data_windows))

wrong_list = []

for data in data_windows:
    if len(data) != 50:
        wrong_list.append(data)

print(len(wrong_list))

50
54648
0


In [72]:
unique_label_list = list(set(label_list))

In [73]:
for a in set(label_list):
    print(a)

standing
descending stairs
lying down on stomach
lying down back
ascending stairs
sitting
miscellaneous movements
normal walking
running
lying down right
lying down on left
shuffle walking


In [74]:
print()

In [75]:
def create_model_offline(input_shape, num_classes):
    model = models.Sequential([
        # CNN layers for spatial feature extraction
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.BatchNormalization(),

        #  # GRU layers
        layers.GRU(128, return_sequences=True, 
                   kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        layers.GRU(64, kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        layers.BatchNormalization(),
        layers.Dropout(0.5),  # Dropout

        # layers.LSTM(256, return_sequences=True, kernel_regularizer=regularizers.l2(0.001)),
        # layers.LSTM(256, kernel_regularizer=regularizers.l2(0.001)),
        # layers.BatchNormalization(),
        # layers.Dropout(0.5),

        # Dense layers for final classification
        layers.Dense(64, activation='relu'),  # First fully connected layer
        layers.Dense(32, activation='relu'),  # Second fully connected layer
        layers.Dense(num_classes, activation='softmax'),  # Output layer
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [76]:
X = np.array([df for df in data_windows])
X = np.stack(X)
y = LabelEncoder().fit_transform(label_list)

# Convert X to float32 and y to categorical (for softmax)
X = X.astype('float32')
y_categorical = to_categorical(y)

In [77]:
overall_dict = {}

In [78]:
for n in label_list:
    if n not in overall_dict:
        overall_dict[n] = 1
    elif n in overall_dict:
        overall_dict[n] += 1
        

In [79]:
overall_dict

{'lying down on left': 7452,
 'sitting': 8694,
 'shuffle walking': 1242,
 'lying down right': 7452,
 'lying down back': 7452,
 'lying down on stomach': 7452,
 'ascending stairs': 1242,
 'standing': 8694,
 'running': 1242,
 'descending stairs': 1242,
 'miscellaneous movements': 1242,
 'normal walking': 1242}

In [80]:
# # Define the Leave-One-Out cross-validator
# loo = LeaveOneOut()

# # Lists to store scores for each fold
# accuracy_scores = []

# # Iterate over each train-test split
# for train_index, test_index in loo.split(X):
#     # Split the data into training and test set for the current fold
#     X_train_fold, X_test_fold = X[train_index], X[test_index]
#     y_train_fold, y_test_fold = y_categorical[train_index], y_categorical[test_index]

#     # Define input_shape and num_classes based on the training data
#     input_shape = X_train_fold.shape[1:]  # (timesteps, features)
#     num_classes = y_train_fold.shape[1]   # number of categories

#     # Create a new model instance (define create_model function as per your model architecture)
#     model = create_model_offline(input_shape, num_classes)

#     # Train the model
#     model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, verbose=0)

#     # Evaluate the model on the test set
#     loss, accuracy = model.evaluate(X_test_fold, y_test_fold, verbose=0)
#     accuracy_scores.append(accuracy)

# # Calculate the average accuracy over all LOO folds
# average_accuracy = np.mean(accuracy_scores)
# print(f'LOO Cross-Validation Accuracy: {average_accuracy:.4f}')

In [81]:
# # Evaluate the model on the test data
# scores = model.evaluate(X_test_fold, y_test_fold, verbose=0)
# accuracy_scores.append(scores[1])  # Assuming that scores[1] is accuracy if you followed the standard compile

# # The final accuracy is the mean of all fold accuracies
# average_accuracy = np.mean(accuracy_scores)
# print(f'LOO Cross-Validation Accuracy: {average_accuracy:.4f}')


In [82]:
def create_model_online(input_shape, num_classes):
    model = models.Sequential([
        # CNN layers for spatial feature extraction
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.BatchNormalization(),
        
        layers.Flatten(),

        #  # GRU layers
        # layers.GRU(128, return_sequences=True, 
        #            kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        # layers.GRU(64, kernel_regularizer=regularizers.l2(0.001)),  # L2 Regularization
        # layers.BatchNormalization(),
        # layers.Dropout(0.5),  # Dropout

        # Dense layers for final classification
        layers.Dense(64, activation='relu'),  # First fully connected layer
        layers.Dense(32, activation='relu'),  # Second fully connected layer
        layers.Dense(num_classes, activation='softmax'),  # Output layer
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [83]:
# Define input_shape and num_classes based on the full dataset
input_shape = X.shape[1:]  # (timesteps, features)
num_classes = y_categorical.shape[1]   # number of categories

In [84]:
test_list = [[0, 0, -1]] * 50
test_input_data = np.array(test_list).reshape(input_shape)


In [85]:
# Create a new model instance using the defined architecture
model_online = create_model_online(input_shape, num_classes)

# Split the dataset into training and validation sets if needed
# If you want to use the full dataset for training, you can skip this step
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train the model
history = model_online.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    verbose=1,
    validation_data=(X_val, y_val)  # Omit this if not using a validation set
)

# Evaluate the model on the validation set if it was used
if 'X_val' in locals():
    val_loss, val_accuracy = model_online.evaluate(X_val, y_val, verbose=1)
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    


# If you have a separate test set, evaluate on it
# test_loss, test_accuracy = model.evaluate([test_input_data], [], verbose=1)
# print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch 1/10
1367/1367 [==============================] - 9s 7ms/step - loss: 0.5646 - accuracy: 0.7786 - val_loss: 0.5480 - val_accuracy: 0.7797
Epoch 2/10
1367/1367 [==============================] - 9s 7ms/step - loss: 0.4323 - accuracy: 0.8171 - val_loss: 0.4073 - val_accuracy: 0.8347
Epoch 3/10
1367/1367 [==============================] - 10s 7ms/step - loss: 0.3892 - accuracy: 0.8303 - val_loss: 0.4307 - val_accuracy: 0.8184
Epoch 4/10
1367/1367 [==============================] - 10s 7ms/step - loss: 0.3557 - accuracy: 0.8426 - val_loss: 0.4303 - val_accuracy: 0.8203
Epoch 5/10
1367/1367 [==============================] - 9s 7ms/step - loss: 0.3394 - accuracy: 0.8472 - val_loss: 0.3309 - val_accuracy: 0.8478
Epoch 6/10
1367/1367 [==============================] - 9s 7ms/step - loss: 0.3203 - accuracy: 0.8556 - val_loss: 0.3116 - val_accuracy: 0.8611
Epoch 7/10
1367/1367 [==============================] - 9s 7ms/step - loss: 0.3044 - accuracy: 0.8609 - val_loss: 0.3390 - val_accurac

In [86]:
result = model_online.predict(np.array([test_input_data]))


1/1 [==============================] - 0s 64ms/step


In [87]:
result

array([[1.48185553e-09, 1.13321306e-07, 4.52649940e-12, 3.42468120e-05,
        9.99807894e-01, 1.57740549e-04, 4.11339123e-08, 1.25057787e-13,
        2.12555060e-11, 8.16906057e-11, 3.62610143e-13, 2.94600108e-11]],
      dtype=float32)

In [90]:
unique_label_list[np.argmax(result)]

'ascending stairs'

In [88]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)

# tflite_model_test = converter.convert()

# with open("model.tflite", "wb") as f:
#     f.write(tflite_model_test)

converter = tf.lite.TFLiteConverter.from_keras_model(model_online)

# Allow for TensorFlow ops that aren't natively supported in TFLite
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]

# # Disable the lowering of tensor list operations
converter._experimental_lower_tensor_list_ops = False

tflite_model_test = converter.convert()

with open(exported_path + "model_cnn.tflite", "wb") as f:
    f.write(tflite_model_test)


INFO:tensorflow:Assets written to: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpc01o14un/assets


INFO:tensorflow:Assets written to: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpc01o14un/assets
2023-11-07 20:04:57.792474: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-11-07 20:04:57.792486: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-11-07 20:04:57.792602: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpc01o14un
2023-11-07 20:04:57.794862: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-11-07 20:04:57.794868: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/gg/10wpd3jj5v7dthydfjzl0gcm0000gn/T/tmpc01o14un
2023-11-07 20:04:57.804281: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-11-07 20:04:57.872153: I tensorflow/cc/saved_model/loader.cc:215] Running initialization